In [1]:
import os
import zipfile
import time
import pickle
import gc

import pandas as pd
import numpy as np
from tqdm import tqdm

from utils import load_pickle, dump_pickle, get_feature_value, feature_spearmanr, feature_target_spearmanr, addCrossFeature, calibration
from utils import raw_data_path, feature_data_path, cache_pkl_path, analyse

from sklearn.metrics import log_loss
import lightgbm as lgb

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit

In [2]:
features=[
         'item_id',
         'item_brand_id',
         'item_city_id',
         'item_price_level',
         'item_sales_level',
         'item_collected_level',
         'item_pv_level',
#          'user_id',
         'user_gender_id',
         'user_age_level',
         'user_occupation_id',
         'user_star_level',
#          'context_id',
         'context_timestamp',
         'context_page_id',
         'shop_id',
         'shop_review_num_level',
         'shop_review_positive_rate',
         'shop_star_level',
         'shop_score_service',
         'shop_score_delivery',
         'shop_score_description',
#          'day',
#          'hour',
#          'minute',
         'category2_label',
         'category3_label',
#2-9
#          'user_gender_id_item_price_level',
#          'user_gender_id_item_sales_level',
#          'user_gender_id_shop_star_level',
#          'user_gender_id_shop_review_num_level',
#          'user_gender_id_shop_review_positive_rate',
         'user_gender_id_category2_label',
         'user_gender_id_category3_label',
#          'user_gender_id_context_page_id',
#          'user_gender_id_hour',
         'user_age_level_item_price_level',
#          'user_age_level_item_sales_level',
#          'user_age_level_shop_star_level',
#          'user_age_level_shop_review_num_level',
#          'user_age_level_shop_review_positive_rate',
         'user_age_level_category2_label',
         'user_age_level_category3_label',
#          'user_age_level_context_page_id',
#          'user_age_level_hour',
         'user_occupation_id_item_price_level',
#          'user_occupation_id_item_sales_level',
#          'user_occupation_id_shop_star_level',
#          'user_occupation_id_shop_review_num_level',
#          'user_occupation_id_shop_review_positive_rate',
         'user_occupation_id_category2_label',
         'user_occupation_id_category3_label',
#          'user_occupation_id_context_page_id',
#          'user_occupation_id_hour',
#          'user_star_level_item_price_level',
#          'user_star_level_item_sales_level',
#          'user_star_level_shop_star_level',
#          'user_star_level_shop_review_num_level',
#          'user_star_level_shop_review_positive_rate',
#          'user_star_level_category2_label',
#          'user_star_level_category3_label',
#          'user_star_level_context_page_id',
#          'user_star_level_hour',
#          'user_gender_id_user_age_level',
#          'user_gender_id_user_occupation_id',
#          'user_gender_id_user_star_level',
#          'user_age_level_user_occupation_id',
#          'user_age_level_user_star_level',
#          'user_occupation_id_user_star_level',
    
    
# category
#          'item_price_level_bin',
#          'item_sales_level_bin',
#          'item_collected_level_bin',
#          'item_pv_level_bin',
#          'shop_review_num_level_bin',
#          'shop_review_positive_rate_bin',
#          'shop_star_level_bin',
#          'shop_score_service_bin',
#          'shop_score_delivery_bin',
#          'shop_score_description_bin',
#          'hour_bin',

         'item_property_topic_k_15',
    
#2-1
#         'user_category2_label_click_day',
#         'user_category3_label_click_day',
#         'user_shop_id_click_day',
#         'user_item_id_click_day',
#         'user_item_brand_id_click_day',
#         'user_context_page_id_click_day',
#         'user_item_price_level_bin_click_day',
#         'user_item_sales_level_bin_click_day',
#         'user_item_property_topic_k_15_click_day',
    
        'user_category2_label_click_hour',
        'user_category3_label_click_hour',
        'user_shop_id_click_hour',
        'user_item_id_click_hour',
        'user_item_brand_id_click_hour',
        'user_context_page_id_click_hour',
        'user_item_price_level_bin_click_hour',
        'user_item_sales_level_bin_click_hour',
        'user_item_property_topic_k_15_click_hour',
    
    
#2-2    
#          'user_id_click_day_mean',
#          'category2_label_click_day_mean',
#          'category3_label_click_day_mean',
#          'shop_id_click_day_mean',
#          'item_id_click_day_mean',
#          'item_brand_id_click_day_mean',
#          'context_page_id_click_day_mean',
    
    
#2-3
        
        'user_click_rank_day',
        'user_click_true_rank_day',
        'user_click_interval_first_day',
        'user_click_interval_last_day',
        'user_click_time_gap_before',
        'user_click_time_gap_after',
    
#         'user_click_rank_global',
#         'user_click_true_rank_global',
#         'user_click_interval_first_global',
#         'user_click_interval_last_global',
#         'user_click_time_gap_before_global',
#         'user_click_time_gap_after_global',
    
        'user_category2_label_click_rank_day',
        'user_category2_label_click_true_rank_day',
        'user_category2_label_first_click_interval_day',
        'user_category2_label_last_click_interval_day',
        'user_category2_label_click_time_gap_before_day',
        'user_category2_label_click_time_gap_after_day',
        'user_category3_label_click_rank_day',
        'user_category3_label_click_true_rank_day',
        'user_category3_label_first_click_interval_day',
        'user_category3_label_last_click_interval_day',
        'user_category3_label_click_time_gap_before_day',
        'user_category3_label_click_time_gap_after_day',
        'user_shop_id_click_rank_day',
        'user_shop_id_click_true_rank_day',
        'user_shop_id_first_click_interval_day',
        'user_shop_id_last_click_interval_day',
        'user_shop_id_click_time_gap_before_day',
        'user_shop_id_click_time_gap_after_day',
        'user_item_id_click_rank_day',
        'user_item_id_click_true_rank_day',
        'user_item_id_first_click_interval_day',
        'user_item_id_last_click_interval_day',
        'user_item_id_click_time_gap_before_day',
        'user_item_id_click_time_gap_after_day',
        'user_item_brand_id_click_rank_day',
        'user_item_brand_id_click_true_rank_day',
        'user_item_brand_id_first_click_interval_day',
        'user_item_brand_id_last_click_interval_day',
        'user_item_brand_id_click_time_gap_before_day',
        'user_item_brand_id_click_time_gap_after_day',
        'user_item_property_topic_k_15_click_rank_day',
        'user_item_property_topic_k_15_click_true_rank_day',
        'user_item_property_topic_k_15_first_click_interval_day',
        'user_item_property_topic_k_15_last_click_interval_day',
        'user_item_property_topic_k_15_click_time_gap_before_day',
        'user_item_property_topic_k_15_click_time_gap_after_day',
    
#         'user_category2_label_click_rank_global',
#         'user_category2_label_click_true_rank_global',
#         'user_category2_label_first_click_interval_global',
#         'user_category2_label_last_click_interval_global',
#         'user_category2_label_click_time_gap_before_global',
#         'user_category2_label_click_time_gap_after_global',
#         'user_category3_label_click_rank_global',
#         'user_category3_label_click_true_rank_global',
#         'user_category3_label_first_click_interval_global',
#         'user_category3_label_last_click_interval_global',
#         'user_category3_label_click_time_gap_before_global',
#         'user_category3_label_click_time_gap_after_global',
#         'user_shop_id_click_rank_global',
#         'user_shop_id_click_true_rank_global',
#         'user_shop_id_first_click_interval_global',
#         'user_shop_id_last_click_interval_global',
#         'user_shop_id_click_time_gap_before_global',
#         'user_shop_id_click_time_gap_after_global',
#         'user_item_id_click_rank_global',
#         'user_item_id_click_true_rank_global',
#         'user_item_id_first_click_interval_global',
#         'user_item_id_last_click_interval_global',
#         'user_item_id_click_time_gap_before_global',
#         'user_item_id_click_time_gap_after_global',
#         'user_item_brand_id_click_rank_global',
#         'user_item_brand_id_click_true_rank_global',
#         'user_item_brand_id_first_click_interval_global',
#         'user_item_brand_id_last_click_interval_global',
#         'user_item_brand_id_click_time_gap_before_global',
#         'user_item_brand_id_click_time_gap_after_global',
#         'user_item_property_topic_k_15_click_rank_global',
#         'user_item_property_topic_k_15_click_true_rank_global',
#         'user_item_property_topic_k_15_first_click_interval_global',
#         'user_item_property_topic_k_15_last_click_interval_global',
#         'user_item_property_topic_k_15_click_time_gap_before_global',
#         'user_item_property_topic_k_15_click_time_gap_after_global',
    
#2-4
        'item_property_topic_0',
        'item_property_topic_1',
        'item_property_topic_2',
        'item_property_topic_3',
        'item_property_topic_4',
        'item_property_topic_5',
        'item_property_topic_6',
        'item_property_topic_7',
        'item_property_topic_8',
        'item_property_topic_9',
        'item_property_topic_10',
        'item_property_topic_11',
        'item_property_topic_12',
        'item_property_topic_13',
        'item_property_topic_14',
        'property_sim',
        'category_predict_rank',

    
    
#2-5
#     交叉特征历史ctr

#         'user_gender_id_item_price_level_smooth_CTR',
#         'user_gender_id_item_sales_level_smooth_CTR',
#         'user_gender_id_shop_star_level_smooth_CTR',
#         'user_gender_id_shop_review_num_level_smooth_CTR',
#         'user_gender_id_shop_review_positive_rate_smooth_CTR',
        'user_gender_id_category2_label_smooth_CTR',
        'user_gender_id_category3_label_smooth_CTR',
        'user_age_level_item_price_level_smooth_CTR',
#         'user_age_level_item_sales_level_smooth_CTR',
#         'user_age_level_shop_star_level_smooth_CTR',
#         'user_age_level_shop_review_num_level_smooth_CTR',
#         'user_age_level_shop_review_positive_rate_smooth_CTR',
        'user_age_level_category2_label_smooth_CTR',
        'user_age_level_category3_label_smooth_CTR',
        'user_occupation_id_item_price_level_smooth_CTR',
#         'user_occupation_id_item_sales_level_smooth_CTR',
#         'user_occupation_id_shop_star_level_smooth_CTR',
#         'user_occupation_id_shop_review_num_level_smooth_CTR',
#         'user_occupation_id_shop_review_positive_rate_smooth_CTR',
        'user_occupation_id_category2_label_smooth_CTR',
        'user_occupation_id_category3_label_smooth_CTR',
#         'user_star_level_item_price_level_smooth_CTR',
#         'user_star_level_item_sales_level_smooth_CTR',
#         'user_star_level_shop_star_level_smooth_CTR',
#         'user_star_level_shop_review_num_level_smooth_CTR',
#         'user_star_level_shop_review_positive_rate_smooth_CTR',
#         'user_star_level_category2_label_smooth_CTR',
#         'user_star_level_category3_label_smooth_CTR',
#         'user_gender_id_user_age_level_smooth_CTR',
#         'user_gender_id_user_occupation_id_smooth_CTR',
#         'user_gender_id_user_star_level_smooth_CTR',
#         'user_age_level_user_occupation_id_smooth_CTR',
#         'user_age_level_user_star_level_smooth_CTR',
#         'user_occupation_id_user_star_level_smooth_CTR',
    
    
#     交叉特征当前小时历史ctr
        'user_gender_id_item_price_level_hour_CTR',
        'user_gender_id_item_sales_level_hour_CTR',
        'user_gender_id_shop_star_level_hour_CTR',
        'user_gender_id_shop_review_num_level_hour_CTR',
        'user_gender_id_shop_review_positive_rate_hour_CTR',
        'user_gender_id_category2_label_hour_CTR',
        'user_gender_id_category3_label_hour_CTR',
        'user_age_level_item_price_level_hour_CTR',
        'user_age_level_item_sales_level_hour_CTR',
        'user_age_level_shop_star_level_hour_CTR',
        'user_age_level_shop_review_num_level_hour_CTR',
        'user_age_level_shop_review_positive_rate_hour_CTR',
        'user_age_level_category2_label_hour_CTR',
        'user_age_level_category3_label_hour_CTR',
        'user_occupation_id_item_price_level_hour_CTR',
        'user_occupation_id_item_sales_level_hour_CTR',
        'user_occupation_id_shop_star_level_hour_CTR',
        'user_occupation_id_shop_review_num_level_hour_CTR',
        'user_occupation_id_shop_review_positive_rate_hour_CTR',
        'user_occupation_id_category2_label_hour_CTR',
        'user_occupation_id_category3_label_hour_CTR',
        'user_star_level_item_price_level_hour_CTR',
        'user_star_level_item_sales_level_hour_CTR',
        'user_star_level_shop_star_level_hour_CTR',
        'user_star_level_shop_review_num_level_hour_CTR',
        'user_star_level_shop_review_positive_rate_hour_CTR',
        'user_star_level_category2_label_hour_CTR',
        'user_star_level_category3_label_hour_CTR',
        'user_gender_id_user_age_level_hour_CTR',
        'user_gender_id_user_occupation_id_hour_CTR',
        'user_gender_id_user_star_level_hour_CTR',
        'user_age_level_user_occupation_id_hour_CTR',
        'user_age_level_user_star_level_hour_CTR',
        'user_occupation_id_user_star_level_hour_CTR',
    
    
#     单特征历史点击与ctr    
         'user_id_smooth_I',
         'user_id_smooth_C',
         'user_id_smooth_CTR',
         'item_id_smooth_I',
         'item_id_smooth_C',
         'item_id_smooth_CTR',
#          'item_brand_id_smooth_I',
#          'item_brand_id_smooth_C',
         'item_brand_id_smooth_CTR',
#          'category2_label_smooth_I',
#          'category2_label_smooth_C',
         'category2_label_smooth_CTR',
#          'category3_label_smooth_I',
#          'category3_label_smooth_C',
         'category3_label_smooth_CTR',
#          'context_page_id_smooth_I',
#          'context_page_id_smooth_C',
#          'context_page_id_smooth_CTR',
#          'shop_id_smooth_I',
#          'shop_id_smooth_C',
         'shop_id_smooth_CTR',
#          'item_sales_level_bin_smooth_I',
#          'item_sales_level_bin_smooth_C',
         'item_sales_level_bin_smooth_CTR',
#          'item_price_level_bin_smooth_I',
#          'item_price_level_bin_smooth_C',
         'item_price_level_bin_smooth_CTR',
#          'item_collected_level_bin_smooth_I',
#          'item_collected_level_bin_smooth_C',
         'item_collected_level_bin_smooth_CTR',
#          'item_pv_level_bin_smooth_I',
#          'item_pv_level_bin_smooth_C',
         'item_pv_level_bin_smooth_CTR',
#          'shop_review_num_level_bin_smooth_I',
#          'shop_review_num_level_bin_smooth_C',
         'shop_review_num_level_bin_smooth_CTR',
#          'shop_review_positive_rate_bin_smooth_I',
#          'shop_review_positive_rate_bin_smooth_C',
         'shop_review_positive_rate_bin_smooth_CTR',
#          'shop_star_level_bin_smooth_I',
#          'shop_star_level_bin_smooth_C',
         'shop_star_level_bin_smooth_CTR',
#          'shop_score_service_bin_smooth_I',
#          'shop_score_service_bin_smooth_C',
         'shop_score_service_bin_smooth_CTR',
#          'shop_score_delivery_bin_smooth_I',
#          'shop_score_delivery_bin_smooth_C',
         'shop_score_delivery_bin_smooth_CTR',
#          'shop_score_description_bin_smooth_I',
#          'shop_score_description_bin_smooth_C',
         'shop_score_description_bin_smooth_CTR',
#          'hour_smooth_I',
#          'hour_smooth_C',
         'hour_smooth_CTR',
    
    
         'user_id_hour_CTR',
         'item_id_hour_CTR',
         'item_brand_id_hour_CTR',
         'category2_label_hour_CTR',
         'category3_label_hour_CTR',
         'context_page_id_hour_CTR',
         'shop_id_hour_CTR',
         'item_sales_level_bin_hour_CTR',
         'item_price_level_bin_hour_CTR',
         'item_collected_level_bin_hour_CTR',
         'item_pv_level_bin_hour_CTR',
         'shop_review_num_level_bin_hour_CTR',
         'shop_review_positive_rate_bin_hour_CTR',
         'shop_star_level_bin_hour_CTR',
         'shop_score_service_bin_hour_CTR',
         'shop_score_delivery_bin_hour_CTR',
         'shop_score_description_bin_hour_CTR',
    
    
#     用户交叉特征历史ctr
        'user_id_item_id_history_I',
        'user_id_item_id_history_C',
#         'user_id_item_id_history_CTR',
        'user_id_item_brand_id_history_I',
        'user_id_item_brand_id_history_C',
#         'user_id_item_brand_id_history_CTR',
        'user_id_category2_label_history_I',
        'user_id_category2_label_history_C',
#         'user_id_category2_label_history_CTR',
        'user_id_category3_label_history_I',
        'user_id_category3_label_history_C',
#         'user_id_category3_label_history_CTR',
        'user_id_shop_id_history_I',
        'user_id_shop_id_history_C',
#         'user_id_shop_id_history_CTR',
#         'user_id_item_sales_level_bin_history_I',
#         'user_id_item_sales_level_bin_history_C',
#         'user_id_item_sales_level_bin_history_CTR',
#         'user_id_item_price_level_bin_history_I',
#         'user_id_item_price_level_bin_history_C',
#         'user_id_item_price_level_bin_history_CTR',

    
#     各种id属性与用户属性交叉历史ctr
        'user_gender_id_shop_id_smooth_CTR',
        'user_gender_id_item_id_smooth_CTR',
        'user_gender_id_item_brand_id_smooth_CTR',
        'user_age_level_shop_id_smooth_CTR',
        'user_age_level_item_id_smooth_CTR',
        'user_age_level_item_brand_id_smooth_CTR',
        'user_occupation_id_shop_id_smooth_CTR',
        'user_occupation_id_item_id_smooth_CTR',
        'user_occupation_id_item_brand_id_smooth_CTR',
#         'user_star_level_shop_id_smooth_CTR',
#         'user_star_level_item_id_smooth_CTR',
#         'user_star_level_item_brand_id_smooth_CTR',
    
    
    
#2-6
         'shop_id_user_gender_id_click_rate',
         'shop_id_user_age_level_click_rate',
         'shop_id_user_occupation_id_click_rate',
#          'shop_id_user_star_level_click_rate',
         'item_id_user_gender_id_click_rate',
         'item_id_user_age_level_click_rate',
         'item_id_user_occupation_id_click_rate',
#          'item_id_user_star_level_click_rate',
         'item_brand_id_user_gender_id_click_rate',
         'item_brand_id_user_age_level_click_rate',
         'item_brand_id_user_occupation_id_click_rate',
#          'item_brand_id_user_star_level_click_rate',
         'category2_label_user_gender_id_click_rate',
         'category2_label_user_age_level_click_rate',
         'category2_label_user_occupation_id_click_rate',
#          'category2_label_user_star_level_click_rate',
         'category3_label_user_gender_id_click_rate',
         'category3_label_user_age_level_click_rate',
         'category3_label_user_occupation_id_click_rate',
#          'category3_label_user_star_level_click_rate',
         'hour_user_gender_id_click_rate',
         'hour_user_age_level_click_rate',
         'hour_user_occupation_id_click_rate',
#          'hour_user_star_level_click_rate',
#          'item_sales_level_bin_user_gender_id_click_rate',
#          'item_sales_level_bin_user_age_level_click_rate',
#          'item_sales_level_bin_user_occupation_id_click_rate',
#          'item_sales_level_bin_user_star_level_click_rate',
         'item_price_level_bin_user_gender_id_click_rate',
         'item_price_level_bin_user_age_level_click_rate',
         'item_price_level_bin_user_occupation_id_click_rate',
#          'item_price_level_bin_user_star_level_click_rate',
    
    
#2-7
#          'user_id_click_day',
#          'item_id_click_day',
#          'item_brand_id_click_day',
#          'category2_label_click_day',
#          'category3_label_click_day',
#          'context_page_id_click_day',
#          'shop_id_click_day',
#          'item_property_topic_k_15_click_day',
    
         'user_id_click_hour',
         'item_id_click_hour',
         'item_brand_id_click_hour',
         'category2_label_click_hour',
         'category3_label_click_hour',
         'context_page_id_click_hour',
         'shop_id_click_hour',
         'item_property_topic_k_15_click_hour',
    
    
#2-8
         'user_item_id_pre_click',
         'user_item_id_continue_click',
         'user_item_brand_id_pre_click',
         'user_item_brand_id_continue_click',
         'user_shop_id_pre_click',
         'user_shop_id_continue_click',
         'user_category2_label_pre_click',
         'user_category2_label_continue_click',
         'user_category3_label_pre_click',
         'user_category3_label_continue_click',
         'user_item_property_topic_k_15_pre_click',
         'user_item_property_topic_k_15_continue_click',
         'user_item_sales_level_bin_pre_click',
         'user_item_sales_level_bin_continue_click',
         'user_item_price_level_bin_pre_click',
         'user_item_price_level_bin_continue_click',
    
    
#2-10  
         'user_category2_label_future_2min',
         'user_category2_label_future_15min',
         'user_category3_label_future_2min',
         'user_category3_label_future_15min',
         'user_shop_id_future_2min',
         'user_shop_id_future_15min',
         'user_item_id_future_2min',
         'user_item_id_future_15min',
         'user_item_brand_id_future_2min',
         'user_item_brand_id_future_15min',
         'user_item_property_topic_k_15_future_2min',
         'user_item_property_topic_k_15_future_15min',
#          'user_item_sales_level_bin_future_2min',
#          'user_item_sales_level_bin_future_15min',
#          'user_item_price_level_bin_future_2min',
#          'user_item_price_level_bin_future_15min'
    
         'shop_id_user_gender_id_click_rate_hour',
         'shop_id_user_age_level_click_rate_hour',
         'shop_id_user_occupation_id_click_rate_hour',
         'shop_id_user_star_level_click_rate_hour',
         'item_id_user_gender_id_click_rate_hour',
         'item_id_user_age_level_click_rate_hour',
         'item_id_user_occupation_id_click_rate_hour',
         'item_id_user_star_level_click_rate_hour',
         'item_brand_id_user_gender_id_click_rate_hour',
         'item_brand_id_user_age_level_click_rate_hour',
         'item_brand_id_user_occupation_id_click_rate_hour',
         'item_brand_id_user_star_level_click_rate_hour',
         'category2_label_user_gender_id_click_rate_hour',
         'category2_label_user_age_level_click_rate_hour',
         'category2_label_user_occupation_id_click_rate_hour',
         'category2_label_user_star_level_click_rate_hour',
         'category3_label_user_gender_id_click_rate_hour',
         'category3_label_user_age_level_click_rate_hour',
         'category3_label_user_occupation_id_click_rate_hour',
         'category3_label_user_star_level_click_rate_hour',
         'item_sales_level_bin_user_gender_id_click_rate_hour',
         'item_sales_level_bin_user_age_level_click_rate_hour',
         'item_sales_level_bin_user_occupation_id_click_rate_hour',
         'item_sales_level_bin_user_star_level_click_rate_hour',
         'item_price_level_bin_user_gender_id_click_rate_hour',
         'item_price_level_bin_user_age_level_click_rate_hour',
         'item_price_level_bin_user_occupation_id_click_rate_hour',
         'item_price_level_bin_user_star_level_click_rate_hour',
    
    
         'user_category2_label_before_2min',
         'user_category2_label_before_15min',
         'user_category2_label_before_1hour',
         'user_category3_label_before_2min',
         'user_category3_label_before_15min',
         'user_category3_label_before_1hour',
         'user_shop_id_before_2min',
         'user_shop_id_before_15min',
         'user_shop_id_before_1hour',
         'user_item_id_before_2min',
         'user_item_id_before_15min',
         'user_item_id_before_1hour',
         'user_item_brand_id_before_2min',
         'user_item_brand_id_before_15min',
         'user_item_brand_id_before_1hour',
#          'user_item_sales_level_bin_before_2min',
#          'user_item_sales_level_bin_before_15min',
#          'user_item_sales_level_bin_before_1hour',
#          'user_item_price_level_bin_before_2min',
#          'user_item_price_level_bin_before_15min',
#          'user_item_price_level_bin_before_1hour',
         'user_item_property_topic_k_15_before_2min',
         'user_item_property_topic_k_15_before_15min',
         'user_item_property_topic_k_15_before_1hour'
         ]
len(features)

301

In [3]:
all_data_path = feature_data_path + 'all_data_all_features_new.pkl'
all_data = load_pickle(all_data_path)

target = 'is_trade'

In [4]:
categorical_feature = [
             'item_price_level',
             'item_sales_level',
             'item_collected_level',
             'item_pv_level',
           
            'user_age_level',
            'user_gender_id',
#             'user_occupation_id',
            'user_star_level',
            'item_property_topic_k_15',
    
            'shop_review_num_level',
            'shop_star_level',
    
#             'category2_label',
#             'category3_label',

    
            'user_click_rank_day',
            'user_category2_label_click_rank_day',
            'user_category3_label_click_rank_day',
            'user_shop_id_click_rank_day',
            'user_item_id_click_rank_day',
            'user_item_brand_id_click_rank_day',
    
# #          'user_item_id_pre_click',
#          'user_item_id_continue_click',
# #          'user_item_brand_id_pre_click',
#          'user_item_brand_id_continue_click',
# #          'user_shop_id_pre_click',
#          'user_shop_id_continue_click',
# #          'user_category2_label_pre_click',
#          'user_category2_label_continue_click',
# #          'user_category3_label_pre_click',
#          'user_category3_label_continue_click',
# #          'user_item_property_topic_k_15_pre_click',
#          'user_item_property_topic_k_15_continue_click',
    
            'category_predict_rank',
    

]
len(categorical_feature)

17

In [5]:
def CustomCV(data,):    
    fold_index_train = data[(data.hour < 11)].index
    fold_index_test = data[data.hour >= 11].index
    
    yield fold_index_train, fold_index_test

In [ ]:
if __name__ == '__main__':


    data = all_data[(all_data.day == 7) & (all_data.is_trade != -1)& (all_data.is_trade != -2)&(all_data.hour>1)]
    data = data.reset_index()
    
    eval_data = data[(data.day == 7) & (data.hour >= 11)]
    eval_set = [(eval_data[features], eval_data[target])]

    print(data.shape,eval_data.shape)
    
    lgb_clf = lgb.LGBMClassifier(objective='binary', device='gpu',  n_jobs=2, silent=False)


#  参数的组合
    lgb_param_grad = {'n_estimators': (4000, ),
                      'learning_rate': (0.03, ),

                      'max_depth': (4,),
                      'num_leaves': (16, 12),
                      #'min_child_samples': (500, 1000),
                      #'min_child_weight': (0.01, 0.1),
                      #'min_split_gain': (0.1, 0.02),
                      

                      'colsample_bytree': (0.8,0.75,),
                      'subsample': (0.8,0.75,),
                      'subsample_freq': (1,),
                      
                      'reg_lambda': (12, 15),
                      
                      'max_bin': (63, ),
                      
                      'gpu_use_dp': (False, ),
                      }

    clf = GridSearchCV(lgb_clf, param_grid=lgb_param_grad, scoring='neg_log_loss',
                       cv=CustomCV(data), n_jobs=4, verbose=1, refit=False, return_train_score=True)

    clf.fit(data[features], data[target],
            feature_name=features,
            categorical_feature=categorical_feature,
            early_stopping_rounds=300, eval_set=eval_set, verbose=50
           )

    print('=====')
    print("Best parameters set found on development set:")
    print(clf.best_params_)

    print('=====')
    print("Best parameters set found on development set:")
    print(clf.best_score_)
    
    dump_pickle(pd.DataFrame(data=clf.cv_results_), '05011-gridsearch-qwc.pkl')


(804307, 515) (126942, 515)
Fitting 1 folds for each of 16 candidates, totalling 16 fits


/home/qwc/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1038: UserWarning: categorical_feature in Dataset is overridden. New categorical_feature is ['category_predict_rank', 'item_collected_level', 'item_price_level', 'item_property_topic_k_15', 'item_pv_level', 'item_sales_level', 'shop_review_num_level', 'shop_star_level', 'user_age_level', 'user_category2_label_click_rank_day', 'user_category3_label_click_rank_day', 'user_click_rank_day', 'user_gender_id', 'user_item_brand_id_click_rank_day', 'user_item_id_click_rank_day', 'user_shop_id_click_rank_day', 'user_star_level']
  warnings.warn('categorical_feature in Dataset is overridden. New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/home/qwc/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1038: UserWarning: categorical_feature in Dataset is overridden. New categorical_feature is ['category_predict_rank', 'item_collected_level', 'item_price_level', 'item_property_topic_k_15', 'item_pv_level'

Training until validation scores don't improve for 300 rounds.
Training until validation scores don't improve for 300 rounds.
Training until validation scores don't improve for 300 rounds.
[50]	valid_0's binary_logloss: 0.229227
Training until validation scores don't improve for 300 rounds.
[50]	valid_0's binary_logloss: 0.229198
[50]	valid_0's binary_logloss: 0.22926
[100]	valid_0's binary_logloss: 0.173281
[100]	valid_0's binary_logloss: 0.173277
[50]	valid_0's binary_logloss: 0.229296
[100]	valid_0's binary_logloss: 0.173337
[150]	valid_0's binary_logloss: 0.165572
[150]	valid_0's binary_logloss: 0.165594
[200]	valid_0's binary_logloss: 0.163814
[100]	valid_0's binary_logloss: 0.173329
[200]	valid_0's binary_logloss: 0.163836
[150]	valid_0's binary_logloss: 0.165586
[250]	valid_0's binary_logloss: 0.163025
[250]	valid_0's binary_logloss: 0.163022
[200]	valid_0's binary_logloss: 0.16382
[150]	valid_0's binary_logloss: 0.165638
[300]	valid_0's binary_logloss: 0.162565
[250]	valid_0's 

/home/qwc/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1038: UserWarning: categorical_feature in Dataset is overridden. New categorical_feature is ['category_predict_rank', 'item_collected_level', 'item_price_level', 'item_property_topic_k_15', 'item_pv_level', 'item_sales_level', 'shop_review_num_level', 'shop_star_level', 'user_age_level', 'user_category2_label_click_rank_day', 'user_category3_label_click_rank_day', 'user_click_rank_day', 'user_gender_id', 'user_item_brand_id_click_rank_day', 'user_item_id_click_rank_day', 'user_shop_id_click_rank_day', 'user_star_level']
  warnings.warn('categorical_feature in Dataset is overridden. New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[2200]	valid_0's binary_logloss: 0.160743
[1800]	valid_0's binary_logloss: 0.160698
[2250]	valid_0's binary_logloss: 0.160751
Training until validation scores don't improve for 300 rounds.
[1850]	valid_0's binary_logloss: 0.160695
[2300]	valid_0's binary_logloss: 0.160759
[1900]	valid_0's binary_logloss: 0.160689
[50]	valid_0's binary_logloss: 0.229264
[2350]	valid_0's binary_logloss: 0.160755
[1950]	valid_0's binary_logloss: 0.160699
Early stopping, best iteration is:
[2096]	valid_0's binary_logloss: 0.160729
[100]	valid_0's binary_logloss: 0.173441


...

[1150]	valid_0's binary_logloss: 0.160842
[2000]	valid_0's binary_logloss: 0.160733
[350]	valid_0's binary_logloss: 0.162356
[1200]	valid_0's binary_logloss: 0.16082
Training until validation scores don't improve for 300 rounds.
[2050]	valid_0's binary_logloss: 0.160741
[400]	valid_0's binary_logloss: 0.16209
[1250]	valid_0's binary_logloss: 0.160803
[50]	valid_0's binary_logloss: 0.229221
Early stopping, best iteration is:
[1787]	valid_0's binary_logloss: 0.160712
[1300]	valid_0's binary_logloss: 0.16079
[450]	valid_0's binary_logloss: 0.161897
[100]	valid_0's binary_logloss: 0.173455
[1350]	valid_0's binary_logloss: 0.16078
[500]	valid_0's binary_logloss: 0.161729
[150]	valid_0's binary_logloss: 0.165716
[1400]	valid_0's binary_logloss: 0.16076
[550]	valid_0's binary_logloss: 0.16161
[1450]	valid_0's binary_logloss: 0.160732
[200]	valid_0's binary_logloss: 0.163929
[600]	valid_0's binary_logloss: 0.1615
[1500]	valid_0's binary_logloss: 0.160733
[250]	valid_0's binary_logloss: 0.16310

/home/qwc/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1038: UserWarning: categorical_feature in Dataset is overridden. New categorical_feature is ['category_predict_rank', 'item_collected_level', 'item_price_level', 'item_property_topic_k_15', 'item_pv_level', 'item_sales_level', 'shop_review_num_level', 'shop_star_level', 'user_age_level', 'user_category2_label_click_rank_day', 'user_category3_label_click_rank_day', 'user_click_rank_day', 'user_gender_id', 'user_item_brand_id_click_rank_day', 'user_item_id_click_rank_day', 'user_shop_id_click_rank_day', 'user_star_level']
  warnings.warn('categorical_feature in Dataset is overridden. New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[650]	valid_0's binary_logloss: 0.16141
[1550]	valid_0's binary_logloss: 0.160711
[300]	valid_0's binary_logloss: 0.162637
Training until validation scores don't improve for 300 rounds.
[700]	valid_0's binary_logloss: 0.16135
[1600]	valid_0's binary_logloss: 0.160706
[350]	valid_0's binary_logloss: 0.162309
[750]	valid_0's binary_logloss: 0.161266
[50]	valid_0's binary_logloss: 0.22935
[1650]	valid_0's binary_logloss: 0.160705
[400]	valid_0's binary_logloss: 0.162069
[800]	valid_0's binary_logloss: 0.16118
[1700]	valid_0's binary_logloss: 0.160712
[100]	valid_0's binary_logloss: 0.17349
[450]	valid_0's binary_logloss: 0.161866
[850]	valid_0's binary_logloss: 0.161133
[1750]	valid_0's binary_logloss: 0.160693
[500]	valid_0's binary_logloss: 0.161747
[150]	valid_0's binary_logloss: 0.165758
[900]	valid_0's binary_logloss: 0.161091
[1800]	valid_0's binary_logloss: 0.160693
[550]	valid_0's binary_logloss: 0.161618
[950]	valid_0's binary_logloss: 0.161057
[200]	valid_0's binary_logloss: 0.1

/home/qwc/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1038: UserWarning: categorical_feature in Dataset is overridden. New categorical_feature is ['category_predict_rank', 'item_collected_level', 'item_price_level', 'item_property_topic_k_15', 'item_pv_level', 'item_sales_level', 'shop_review_num_level', 'shop_star_level', 'user_age_level', 'user_category2_label_click_rank_day', 'user_category3_label_click_rank_day', 'user_click_rank_day', 'user_gender_id', 'user_item_brand_id_click_rank_day', 'user_item_id_click_rank_day', 'user_shop_id_click_rank_day', 'user_star_level']
  warnings.warn('categorical_feature in Dataset is overridden. New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[1600]	valid_0's binary_logloss: 0.160817
[800]	valid_0's binary_logloss: 0.161192
[1200]	valid_0's binary_logloss: 0.16093
Training until validation scores don't improve for 300 rounds.
[1650]	valid_0's binary_logloss: 0.160809
[850]	valid_0's binary_logloss: 0.161149
[1250]	valid_0's binary_logloss: 0.160921
[50]	valid_0's binary_logloss: 0.229282
[1700]	valid_0's binary_logloss: 0.160809
[1300]	valid_0's binary_logloss: 0.160899
[900]	valid_0's binary_logloss: 0.161109
[1750]	valid_0's binary_logloss: 0.160806
[100]	valid_0's binary_logloss: 0.173484
[950]	valid_0's binary_logloss: 0.161077
[1350]	valid_0's binary_logloss: 0.160888
[1800]	valid_0's binary_logloss: 0.160797
[150]	valid_0's binary_logloss: 0.165725
[1000]	valid_0's binary_logloss: 0.161045
[1400]	valid_0's binary_logloss: 0.160867
[1850]	valid_0's binary_logloss: 0.160797
[1050]	valid_0's binary_logloss: 0.161017
[200]	valid_0's binary_logloss: 0.163938
[1900]	valid_0's binary_logloss: 0.160798
[1450]	valid_0's binary

In [2]:
gird = load_pickle('05011-gridsearch-qwc.pkl')


pd.DataFrame(data=clf.cv_results_)[['rank_test_score', 'mean_test_score', 'mean_train_score', 'param_reg_lambda', 'param_num_leaves', 'param_colsample_bytree', 
#                                     'param_min_split_gain','param_subsample_freq',
                                    'param_max_bin', 'param_gpu_use_dp']]

FileNotFoundError: [Errno 2] No such file or directory: '05011-gridsearch-qwc.pkl'